# Developped Framework for High DEL Period Analysis.



# Load the model

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os

In [ ]:
#Specify project_path
project_path = r'your_path_to_the_project_folder'

# Load the model
from py_nw2.mlflow_functions import load_model
from py_nw2.model import custom_loss

model_folder_path = os.path.join(project_path, 'models') # path of the folder where the model is saved
model_name = 'DEL_NRT_PB_Mtn_1' # name of the model
model_path = os.path.join(model_folder_path, model_name) # path of the model
model = load_model(dir_path=model_path, custom_objects={'custom_loss':custom_loss}) # load the model

features_names = list(model.info['input_features']) # get the features names

# Load the data



In [ ]:
# Load the data for testing
data_folder_path = os.path.join(project_path, 'data') # path of the folder where the data is saved
input_data = pd.read_parquet(os.path.join(data_folder_path, 'normalized2', 'turbine1_normalized_data.parquet')) # load the data
turbine_x = 'turbine 1' # name of the turbine

# Print the period of the data
print(turbine_x,'data shape :',input_data.shape)
print(turbine_x,'period :',input_data.index[0],'-',input_data.index[-1])

input_data.head()

# Make the DEL predictions (output)

In [ ]:
#make the predictions
predictions_turbine = pd.DataFrame({'DEL': model.predict(input_data).flatten()}, index=input_data.index)

# combine input data and output  
full_data = pd.concat([input_data, predictions_turbine], axis=1)

## Plot of the DEL

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,8))
plt.plot(predictions_turbine, label=turbine_x)
plt.title('DEL predictions')
plt.legend(fontsize='large')  # Increase the font size of the legend
plt.ylabel('DEL [Nm]')
plt.xlabel('Time')
plt.grid()
plt.show()



# Selection of reference samples for Shapley Values analysis

In [ ]:
'''
Function to select reference points based on 3 criteria:
- DEL < threshold_max
- mean_NRT_rpm > threshold_rpm_power
- mean_NRT_power > threshold_rpm_power

To have the rated conditions

number_of_sample : number of reference points to select
'''


def select_ref_point(df,threshold_max,threshold_rpm_power,number_of_sample):


    threshold_max = threshold_max*1000000 # convert to Nm because the DEL order of magnitude is 1e6


    df = df[(df['DEL'] < threshold_max)]
    print('Number of points after DEL selection :',df.shape[0])

    df = df[df['mean_NRT_rpm'] > threshold_rpm_power]
    df = df[df['mean_NRT_power'] > threshold_rpm_power]
    print('Number of points after rpm and power selection :',df.shape[0])
    
   
    ref_sample = df.sample(n=number_of_sample, random_state=2)
    
    return ref_sample

In [ ]:
# parameters 

threshold_max = 1
number_of_sample = 10
threshold_rpm = 0.9


reference_samples = select_ref_point(full_data,threshold_max,threshold_rpm,number_of_sample)


plt.figure(figsize=(20,8))
plt.scatter(full_data.index, full_data['DEL'], label=turbine_x,marker='o', s=1.2, alpha=1)
plt.scatter(reference_samples.index, reference_samples['DEL'], color='red', label='ref point')
plt.title('DEL predictions')
plt.legend(fontsize='large')  # Increase the font size of the legend
plt.ylabel('DEL [Nm]')
plt.xlabel('Time')
plt.grid()
plt.show()


# Selection of the samples at High DEL periods for Shapley Values analysis

In [ ]:
# Plotting

plt.figure(figsize=(20, 8))
plt.scatter(full_data.index, full_data['DEL'], label=turbine_x,s = 2)
plt.title('DEL predictions')
#plt.legend(fontsize='large')
plt.ylabel('DEL [Nm]')
plt.xlabel('Time')
plt.xticks(rotation='vertical')
plt.grid()
plt.show()


In [ ]:
# Selecting the period where High DEL are predicted

start_date1 = '2022-12-25'
end_date1 = '2022-12-31'

start_date2 = '2023-01-07'
end_date2 = '2023-01-15'

start_date3 = '2023-01-28'
end_date3 = '2023-02-02'

start_date4 = '2023-02-25'
end_date4 = '2023-02-27'


start_dates = [start_date1,start_date2,start_date3,start_date4]
end_dates = [end_date1,end_date2,end_date3,end_date4]

number_of_period = len(start_dates)

num_points = 10 # Number of points to select for each period

threshold = 1.5*10**6 # Minimum Threshold for DEL


# Dictionary to hold selected points for each division
selected_points_for_shap = {}


# Selecting the points for each period
for i in range(number_of_period):
    full_data_selected = full_data.loc[start_dates[i]:end_dates[i]]   
    full_data_selected = full_data_selected[full_data_selected['DEL'] > threshold] 
    selected_points_for_shap[i] = full_data_selected.sample(n=num_points, random_state=42)


# Plotting
plt.figure(figsize=(20, 8))
plt.scatter(full_data.index, full_data['DEL'], label=turbine_x,s=   1.5)
for i in range(number_of_period):
    color = 'red' if i % 2 == 0 else 'yellow'  # Alternating colors
    plt.scatter(selected_points_for_shap[i].index, selected_points_for_shap[i]['DEL'], color=color, label='Selected points')

plt.title('DEL predictions')
plt.legend(fontsize='large')
plt.ylabel('DEL [Nm]')
plt.xlabel('Time')
plt.grid()
plt.show()

# Inputs Vizualization

In [ ]:

interesting_features = ['mean_NRT_windspeed','std_NRT_windspeed',
                        'mean_NRT_yaw','mean_NRT_power','mean_NRT_pitch',
                        'mean_NRT_rpm','mean_NRT_winddirection',
                        'mean_NRT_NAC_ACC_Z','rms_NRT_NAC_ACC_FA',
                        'rms_NRT_NAC_ACC_SS','rms_NRT_NAC_ACC_Z',
                        'mean_NRT_ti','rms1p_NRT_NAC_ACC_Z',
                        'rms1p_NRT_NAC_ACC_FA','rms1p_NRT_NAC_ACC_SS']

for feature in enumerate(interesting_features):

    plt.figure(figsize=(20,8))
    plt.plot(full_data[feature[1]], label=turbine_x)
    plt.scatter(reference_samples.index, reference_samples[feature[1]], color='red', label='ref point', zorder=5)
    plt.legend()
    plt.xlabel('time', fontsize=14)
    plt.ylabel(feature[1], fontsize=14)


# Shapley Values Analysis

Background samples selection is performed to approximate the Shapley Values. Increasing the number of samples enhances the accuracy of the approximation. However, there is a trade-off between precision and computational time. A recommended sample size is 400, chosen randomly from the dataset.

In [ ]:
import shap
shap.initjs()

# Summarize the background dataset using shap.sample

n = 400 # number of samples to summarize the background data
background_samples = shap.sample(input_data, n) # summarize the background data

## Shapley values for the reference Samples

In [ ]:
# Computing the shap values

explainer = shap.KernelExplainer(model.predict, background_samples)
shap_values_ref = explainer.shap_values(reference_samples[features_names])


### Bar plot of the reference period

In [ ]:
shap_exp_ref = shap.Explanation(values=shap_values_ref[0],base_values=explainer.expected_value, feature_names=features_names)
Number_of_ft_plot = 10 # Number of features to plot
shap.plots.bar(shap_exp_ref.mean(0),max_display=Number_of_ft_plot)

### Mean Values of the features

Print the mean values of the features ordered by their importance

In [ ]:
import numpy as np

# Calculate the absolute sum of SHAP values for each feature
shap_abs_sum = np.abs(shap_values_ref[0].sum(axis=0))

# Sort the features based on their absolute sum of SHAP values in descending order
sorted_features = sorted(zip(features_names, shap_abs_sum), key=lambda x: x[1], reverse=True)

# Create a list of the most important features
most_important_features = [feature for feature, _ in sorted_features]


# Calculate the average value of each feature for the reference_samples
average_values = np.mean(reference_samples[features_names], axis=0)

# Sort features and average values based on importance list
sorted_features_names = [feature for feature in most_important_features if feature in features_names]
sorted_indices = [features_names.index(feature) for feature in sorted_features_names]
sorted_average_values = average_values[sorted_indices]
for feature in most_important_features:
    value = average_values[feature]
    value = round(value, 2)
    print(f"{feature}: {value}")
  



## Shapley values for the periods of interest

In [ ]:

shap_values_all = {} # Dictionary to hold the shap values for each period

for i in range(number_of_period):

    # Computing the shap values for each period

    explainer = shap.KernelExplainer(model.predict, background_samples)
    shap_values_all[i] = explainer.shap_values(selected_points_for_shap[i][features_names])


In [ ]:
period_number = 3 # Period number to plot the bar plot (the period count starts from 0)
shap_exp = shap.Explanation(values=shap_values_all[period_number][0], base_values=explainer.expected_value,feature_names=features_names)
print('Shape of the shap value matrix',shap_exp.shape)
shap.plots.bar(shap_exp.mean(0),max_display=21)


### Mean Values of the features in the selected period

Print the mean values of the features ordered by their importance

In [ ]:
# Calculate the absolute sum of SHAP values for each feature
shap_abs_sum = abs(shap_values_all[period_number][0].sum(axis=0))

# Sort the features based on their absolute sum of SHAP values in descending order
sorted_features = sorted(zip(features_names, shap_abs_sum), key=lambda x: x[1], reverse=True)

# Create a list of the most important features
most_important_features = [feature for feature, _ in sorted_features]


# Calculate the average value of each feature for the reference_samples
average_values = np.mean(selected_points_for_shap[period_number][features_names], axis=0)

# Sort features and average values based on importance list
sorted_features_names = [feature for feature in most_important_features if feature in features_names]
sorted_indices = [features_names.index(feature) for feature in sorted_features_names]
sorted_average_values = average_values[sorted_indices]
for feature in most_important_features:
    value = average_values[feature]
    value = round(value, 2)
    print(f"{feature}: {value}")


